In [1]:
#import libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.linear_model import LogisticRegression

In [19]:
def wrangle(file):
    df = pd.read_csv(file)
    cols_to_drop =['Adm5', 'Adm6', 'Geo1', 'econ4']
   
    
    df.drop(columns = cols_to_drop, inplace=True)
    #mask = df['Class'].str.contains('Recurrent')
    #df = df[mask]
    return df


In [20]:
df = wrangle('kenya-boost-2006-2018.csv')
df.head()

/tmp/ipykernel_27302/1159460004.py:2: DtypeWarning: Columns (0,1,2,3,4,5,6,7,9,10,11,12,15,16,17,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


,Year,Class,Adm1,Adm2,Adm3,Adm4,Econ0,econ1,econ2,econ3,FndSrce1,function1,function2,Estimates,Revised,Executed


In [18]:
columns = df.columns
for col in columns:
    print(f'Value counts for {col} column')
    try:
        print(df[col].value_counts())
    except Exception as e:
        print(f"Error in column '{col}': {e}")
    print('------------------')

Value counts for Year column
Year
2007/8     38585
2012/13    36485
2011/12    34452
2009/10    33022
2008/9     32181
2010/11    31438
2006/7     31379
2005/6     26199
Name: count, dtype: int64
------------------
Value counts for Class column
Class
Development    162030
Recurrent      101711
Name: count, dtype: int64
------------------
Value counts for Adm1 column
Adm1
Constituency Fund    136022
Central              102027
 Central              13688
Local Authorities     11928
Social security          76
Name: count, dtype: int64
------------------
Value counts for Adm2 column
Adm2
104 Ministry of Foreign Affairs                                              9698
119 Ministry of Livestock Development                                        7411
101 Ministry of State for Provincial Administration and Internal Security    7042
131 Ministry of Education                                                    6471
110 Ministry of Agriculture                                                  53